In [1]:
# Load the Drive helper and mount
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive/USED/Humana')
os.listdir()

['Temp',
 'Humana Share',
 'Data',
 'Humana EDA.ipynb',
 'Preprocessing_v1.ipynb',
 'Model',
 'Preprocessing_v2.ipynb',
 'Zip Encoding.ipynb',
 'columns_type.csv',
 'catboost_info',
 'Fast_Check_Feature_Selection.ipynb',
 'Further_Selection_With_CatBosst.ipynb',
 'Pycaret_1.ipynb',
 'logs.log',
 'Modeling.ipynb',
 'Catboost.ipynb',
 'tab_cat_trails.csv',
 'Tabnet.ipynb',
 'tab_1.npy',
 'subset_test_label_2.csv',
 'subset_test_label_3.csv',
 'subset_test_label_4.csv',
 'subset_test_label_0.csv',
 'subset_test_label_1.csv',
 'pdp.pdf',
 'ice.pdf',
 'pdp_age.pdf',
 'pdp_hum_region.pdf',
 'pdp_atlas_foodinsec_13_15.pdf',
 'ice_age.pdf',
 'ice_hum_region.pdf',
 'ice_atlas_foodinsec_13_15.pdf',
 'pdp_race_cd_isnull.pdf',
 'ice_race_cd_isnull.pdf',
 'race_cd_null.pdf',
 'fea_imp.csv',
 'ice_est_age.pdf',
 'ice_atlas_vlfoodsec_13_15.pdf',
 'ice_cms_risk_adjustment_factor_a_amt.pdf',
 'pdp_est_age.pdf',
 'pdp_atlas_vlfoodsec_13_15.pdf',
 'pdp_cms_risk_adjustment_factor_a_amt.pdf',
 'Copy of mod

# stats encoding

In [5]:
import pandas as pd 
df_zip_train=pd.read_csv('Data/2021_Competition_Training.csv',usecols=['zip_cd','covid_vaccination'])
df_zip_train['covid_vaccination']=df_zip_train['covid_vaccination']!='no_vacc'
df_zip_train['covid_vaccination']=df_zip_train['covid_vaccination'].astype(int)

df_stat=df_zip_train.groupby('zip_cd',as_index=False).agg(['mean','std','skew'])

df_stat.sum()

covid_vaccination  mean    15204.270009
                   std     22510.717406
                   skew    88695.111660
dtype: float64

In [6]:
import gc
gc.collect()

df_zip_train=df_zip_train.merge(df_stat,on='zip_cd')

df_zip_train.columns=['y','zip_cd','zip_enc_mean','zip_enc_std','zip_enc_skew']
df_zip_train=df_zip_train[['zip_enc_mean','zip_enc_std','zip_enc_skew']]
display(df_zip_train)

df_zip_train.to_csv('Data/train_zip_enc.csv',index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,zip_enc_mean,zip_enc_std,zip_enc_skew
0,0.4,0.547723,0.608581
1,0.4,0.547723,0.608581
2,0.4,0.547723,0.608581
3,0.4,0.547723,0.608581
4,0.4,0.547723,0.608581
...,...,...,...
974837,0.0,NaN,NaN
974838,0.0,NaN,NaN
974839,0.0,NaN,NaN
974840,0.0,NaN,NaN


In [7]:
import pandas as pd 
df_zip_test=pd.read_csv('Data/2021_Competition_Holdout.csv',usecols=['zip_cd'])

import gc
gc.collect()

df_zip_test=df_zip_test.merge(df_stat,on='zip_cd',how='left')

df_zip_test.columns=['zip_cd','zip_enc_mean','zip_enc_std','zip_enc_skew']
df_zip_test=df_zip_test[['zip_enc_mean','zip_enc_std','zip_enc_skew']]
display(df_zip_test)

df_zip_test.to_csv('Data/test_zip_enc.csv',index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,zip_enc_mean,zip_enc_std,zip_enc_skew
0,0.161017,0.369114,1.868407
1,0.162162,0.373684,1.911471
2,0.187500,0.396558,1.681134
3,0.000000,0.000000,NaN
4,0.250000,0.440959,1.221116
...,...,...,...
525153,0.133333,0.341839,2.194015
525154,0.244444,0.434613,1.230711
525155,0.333333,0.516398,0.968246
525156,0.000000,0.000000,0.000000


# clustering

In [8]:
import pandas as pd 
df_zip_train=pd.read_csv('Data/2021_Competition_Training.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,8,9,11,13,16,20,26,28,29,31,33,51,55,58,62,64,66,68,75,85,102,124,127,131,132,135,160,174,180,187,192,202,209,210,211,215,220,230,234,240,243,247,251,255,261,285,293,297,300,305,306,309,323,334,344,345,352,353,355,359) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
import numpy as np 
cluster_cols=[]
for col in df_zip_train.columns:
    if df_zip_train[col].dtype!='O':
        cluster_cols.append(col)
        avg=df_zip_train[col].mean()
        df_zip_train[col]=df_zip_train[col].fillna(avg)

In [10]:
import gc 
from sklearn.cluster import MiniBatchKMeans

gc.collect()

kmeans = MiniBatchKMeans(n_clusters=50,
                         random_state=0,
                         batch_size=32,
                         max_iter=10).fit(df_zip_train[cluster_cols])

In [11]:
df_zip_train['cluster']=kmeans.labels_
df_zip_train=df_zip_train[['zip_cd','cluster']]
df_zip_train

,zip_cd,cluster
0,52161,16
1,55415,25
2,29450,42
3,30108,34
4,93569,21
...,...,...
974837,66182,23
974838,43401,45
974839,32122,33
974840,43737,45


In [12]:
df_zip_train['count']=1
df_pivot=df_zip_train.groupby(['zip_cd','cluster'],as_index=False).count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_pivot=df_pivot.pivot(index='zip_cd', columns='cluster', values='count')
cols_sum=df_pivot.sum(axis=1)
for col in df_pivot.columns:
    df_pivot[col]/=cols_sum

In [14]:
df_zip_train.merge(df_pivot.reset_index(),on='zip_cd')[list(range(50))].to_csv('Data/train_zip_enc_cluster.csv',index=False)

In [15]:
df_zip_test=pd.read_csv('Data/2021_Competition_Holdout.csv',usecols=['zip_cd'])
df_zip_test.merge(df_pivot.reset_index(),on='zip_cd',how='left')[list(range(50))].to_csv('Data/test_zip_enc_cluster.csv',index=False)